In [9]:
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize
import json


In [10]:
pd.options.display.max_columns=200
pd.options.display.max_rows=100

In [11]:
token_id = 'Token f643c4cfb329a249febbecca90efa2736acf39ce'
'''
response = requests.get(
    'https://api.case.law/v1/cases/435800/?full_case=true',
    headers={'Authorization': token_id}
)
'''

"\nresponse = requests.get(\n    'https://api.case.law/v1/cases/435800/?full_case=true',\n    headers={'Authorization': token_id}\n)\n"

In [12]:
def search_query(page_size='10000',
                 search='car+accident',
                 full_case='true',
                 jurisdiction = 'cal',
                 decision_date_min='1950-01-01',
                 decision_date_max='2020-12-31'
                 ):
    url = 'https://api.case.law/v1/cases/?page_size='+page_size+\
    '&search='+search+'&full_case='+full_case+'&jurisdiction='+jurisdiction+\
    '&decision_date_min='+decision_date_min +\
    '&decision_date_max='+decision_date_max +\
    '&body_format=text&ordering=relevance'
    return url
    

In [14]:
def pull_data(token_id,
              page_size='10000',
              search='car+accident',
              full_case='true',
              jurisdiction = 'cal',
              decision_date_min='1950-01-01',
              decision_date_max='2020-12-31'):
    # get url
    url = search_query(page_size,
                 search,
                 full_case,
                 jurisdiction,
                 decision_date_min,
                 decision_date_max)
    
    response = requests.get(url,headers={'Authorization': token_id})
    
    return response

## 1. Load cases and save it

In [ ]:
# pull data
response = pull_data(token_id)

In [ ]:
# write to json and save in drive
with open('car_accident.json', 'wb') as outf:
    outf.write(response.content)

## 2. Open the cases

In [20]:
# load json
with open('car_accident.json') as f:
    test_dic = json.load(f)

In [21]:
len(test_dic['results'])

5059

## 3. Convert the json to dataframe for easy reading and modeling

In [23]:
a = test_dic['results']
df = json_normalize(a)

In [24]:
df.head(2)

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,cites_to,frontend_url,frontend_pdf_url,preview,last_updated,volume.url,volume.volume_number,volume.barcode,reporter.url,reporter.full_name,reporter.id,court.url,court.name_abbreviation,court.slug,court.id,court.name,jurisdiction.id,jurisdiction.name_long,jurisdiction.url,jurisdiction.slug,jurisdiction.whitelisted,jurisdiction.name,analysis.word_count,analysis.sha256,analysis.ocr_confidence,analysis.char_count,analysis.pagerank.percentile,analysis.pagerank.raw,analysis.cardinality,analysis.simhash,casebody.status,casebody.data.head_matter,casebody.data.corrections,casebody.data.parties,casebody.data.attorneys,casebody.data.opinions,casebody.data.judges,casebody.data
0,191031,https://api.case.law/v1/cases/191031/,"JOSE D. MARQUEZ, Plaintiff and Appellant, v. E...",Marquez v. Enter. Rent-A-Car,1997-02-07,No. B096993,319,324,"[{'type': 'official', 'cite': '53 Cal. App. 2d...","[{'cite': '278 Cal.Rptr. 533', 'category': 're...",https://cite.case.law/cal-app-2d/53/319/,https://cite.case.law/pdf/191031/Marquez%20v.%...,[ENTERPRISE RENT-A-<em class='search_highlight...,2021-05-29T02:04:42.932697+00:00,https://api.case.law/v1/volumes/32044038629440/,53,32044038629440,https://api.case.law/v1/reporters/334/,"California Appellate Reports, Fourth Series",334,https://api.case.law/v1/courts/cal-ct-app-1/,Cal. Ct. App.,cal-ct-app-1,8799,Court of Appeal of the State of California,30,California,https://api.case.law/v1/jurisdictions/cal/,cal,False,Cal.,1250,eb8ce1c060350dd6268e04352a9dd236022cd56963c627...,0.924,7671,0.007619,4.076307e-08,463,1:186b8601d0e32a46,ok,"[No. B096993.\nSecond Dist., Div. Four.\nFeb. ...",,"[JOSE D. MARQUEZ, Plaintiff and Appellant, v. ...",[],"[{'author': 'EPSTEIN, Acting P. J.', 'text': '...",[],NaN
1,6053508,https://api.case.law/v1/cases/6053508/,"JESUS FLORES et al., Plaintiffs and Appellants...",Flores v. Enterprise Rent-A-Car Co.,2010-09-28,No. B215105,1055,1073,"[{'type': 'official', 'cite': '188 Cal. App. 4...","[{'cite': '134 P. 704', 'category': 'reporters...",https://cite.case.law/cal-app-4th/188/1055/,https://cite.case.law/pdf/6053508/Flores%20v.%...,[ENTERPRISE RENT-A-<em class='search_highlight...,2021-05-28T19:25:40.401243+00:00,https://api.case.law/v1/volumes/32044132278433/,188,32044132278433,https://api.case.law/v1/reporters/334/,"California Appellate Reports, Fourth Series",334,https://api.case.law/v1/courts/cal-ct-app-1/,Cal. Ct. App.,cal-ct-app-1,8799,Court of Appeal of the State of California,30,California,https://api.case.law/v1/jurisdictions/cal/,cal,False,Cal.,6458,df2b4896d8fe589f5f27f598b8d438d47d4fe1a14374f1...,0.908,40450,0.175458,4.076307e-08,1285,1:29c8571457759ad9,ok,"[No. B215105.\nSecond Dist., Div. Four.\nSept....",,"[JESUS FLORES et al., Plaintiffs and Appellant...","[Counsel, Steven B. Stevens for Plaintiffs and...","[{'author': 'WILLHITE, Acting P. J.', 'text': ...",[],NaN


## 4. Look at the full text

In [26]:
# this is the full text - keep! important
df['casebody.data.opinions'][0][0]['text']

'Opinion\nEPSTEIN, Acting P. J.\nWe consider an appeal from a grant of summary judgment for respondent Enterprise Rent-A-Car. Appellant sued respondent and others for injuries stemming from a 1991 automobile accident with Michael Melcher. At the time of the accident, Melcher was driving a car rented from Enterprise by Martin Sanchez. In granting the summary judgment, the court found that Enterprise’s rental agreement did not, either expressly or impliedly, permit Melcher to drive the vehicle. We affirm.\nFactual and Procedural Summary\nIn 1991, Martin Sanchez rented a car from respondent Enterprise Rent-A-Car. His rental agreement provided a space for Sanchez to request permission from respondent to allow other people to drive the car. At the time of his rental, someone had typed the words “no others” into this space. Sanchez initialed this part of the agreement and signed the completed form.\nOn June 8, 1991, the rental car, driven by Michael Melcher, was involved in an accident with 

In [27]:
df['preview'][1]

["ENTERPRISE RENT-A-<em class='search_highlight'>CAR</em> COMPANY et al., Defendants and Respondents.\nCounsel\nSteven B.",
 "Hurst for Defendant and Respondent Enterprise Rent-A-<em class='search_highlight'>Car</em> Company.\nSelman Breitman, Paul E.",
 "Capabianco for Defendant and Respondent Enterprise Rent-A-<em class='search_highlight'>Car</em> Company of Los Angeles.",
 "driven by Dederer at the time of the <em class='search_highlight'>accident</em>.",
 "The <em class='search_highlight'>accident</em> occurred when Dederer, driving the vehicle, looked down to adjust his stereo and the vehicle",
 "Dederer was not under the influence of alcohol at the time of the <em class='search_highlight'>accident</em>, and was not charged with",
 "Four days after renting the vehicle, the renter was involved in an <em class='search_highlight'>accident</em> in which several persons",
 "<em class='search_highlight'>accident</em>."]

In [22]:
# 10 example - car accident, 1950 - 2020, ca, 10 cases
'''
token_id = 'Token f643c4cfb329a249febbecca90efa2736acf39ce'
response = requests.get('https://api.case.law/v1/cases/?\
page_size=10&search=car+accident&full_case=true&jurisdiction=cal&\
decision_date_min=1950-01-01&decision_date_max=2020-12-31&\
body_format=text&ordering=relevance',
                       headers={'Authorization': token_id}
                       )
'''

"\ntoken_id = 'Token f643c4cfb329a249febbecca90efa2736acf39ce'\nresponse = requests.get('https://api.case.law/v1/cases/?page_size=10&search=car+accident&full_case=true&jurisdiction=cal&decision_date_min=1950-01-01&decision_date_max=2020-12-31&body_format=text&ordering=relevance',\n                       headers={'Authorization': token_id}\n                       )\n"